In [5]:
import torch
from utils import *

l1 = one_hot_encoding(3)
l2 = one_hot_encoding(4)
l = [l1,l2]
print(l)

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [9]:
l = torch.Tensor(l)
print(torch.max(l,1).indices)

tensor([3, 4])


In [1]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# 正解シュート方向: [①左端、②中央左、③中央右、④右端 、①左端]
true = np.array([0, 1, 2, 3, 0])

# 推定シュート方向: [②中央左、②中央左、③中央右、④右端 、①左端]
pred = np.array([1, 1, 2, 3, 0])

epsilon = 1e-6

# weights='quadratic'オプションで重みを二乗で計算
kappa_score = cohen_kappa_score(true, pred, weights='quadratic')

print(f'kappa_score = {kappa_score}')
print(f'kappa_loss = {np.log(1 - kappa_score + epsilon)}')

kappa_score = 0.9180327868852459
kappa_loss = -2.5014237518136304


In [1]:
import torch.nn as nn
import torch

In [7]:
loss = nn.MSELoss()
input_ = torch.rand(3,5,requires_grad=True)
target = torch.rand(3,5)
print(input_,target)
output = loss(input_,target)
output.backward()

tensor([[0.5345, 0.9346, 0.8807, 0.5206, 0.7228],
        [0.2893, 0.4710, 0.2705, 0.7708, 0.6866],
        [0.4907, 0.7540, 0.5059, 0.6145, 0.4550]], requires_grad=True) tensor([[0.8402, 0.1270, 0.8526, 0.4140, 0.2325],
        [0.6142, 0.1406, 0.8921, 0.3674, 0.6065],
        [0.8877, 0.7118, 0.7077, 0.1020, 0.9519]])
tensor(0.1652, grad_fn=<MseLossBackward>)


In [9]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input_ = torch.randn(3,requires_grad=True)
target = torch.empty(3).random_(2)
print(input_)
print(target)
output = loss(m(input_),target)
output.backward()
print(output)

tensor([-0.1744, -0.6964,  0.2006], requires_grad=True)
tensor([1., 0., 1.])
tensor(0.5955, grad_fn=<BinaryCrossEntropyBackward>)


In [14]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

class KappaLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, outputs, targets):
        '''損失関数を計算

        Parameters
        ------------------
        outputs : モデルの出力
        targets : 正解ラベル
        '''
        outputs_ = outputs.detach().cpu().numpy()
        targets_ = targets.detach().cpu().numpy()
        kappa = cohen_kappa_score(targets_,outputs_,weights='quadratic')
        epsilon = 1e-6
        k = np.log(1-kappa-epsilon)
        return torch.from_numpy(k)

In [20]:
loss = KappaLoss()
input_ = torch.randn(3,requires_grad=True)
target = torch.empty(3).random_(2)
print(input_)
print(target)
output = loss(m(input_),target)
output.backward()
print(output)

tensor([0., 1., 1.], grad_fn=<CopySlices>)
tensor([0., 1., 1.])


ValueError: Classification metrics can't handle a mix of binary and continuous targets